In [1]:
!pip install openai python-dotenv tqdm


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import os
import json
import time
from getpass import getpass
from pathlib import Path

import pandas as pd
from tqdm.auto import tqdm
from openai import OpenAI


In [21]:
# Masukkan API key DeepSeek kamu (tidak tampil di layar, tidak tersimpan di file)
DEEPSEEK_API_KEY = getpass("Masukkan DEEPSEEK_API_KEY: ")

client = OpenAI(
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com",
)

# Tes singkat
resp = client.chat.completions.create(
    model="deepseek-chat",
    messages=[{"role": "user", "content": "Halo, ini test dari FP Qur'an IR ."}],
    max_tokens=20,
)
print("Respon test:", resp.choices[0].message.content)


Masukkan DEEPSEEK_API_KEY: ··········
Respon test: Halo! Selamat datang! 😊

Saya siap membantu Anda dengan berbagai hal terk


In [22]:
# SESUAIKAN PATH INI DENGAN LOKASI FILE KAMU
PATH_TAFSIR = "/content/drive/MyDrive/fp_quran/tafsir_clean.csv"

PATH_TAFSIR = Path(PATH_TAFSIR)
print("File ada?", PATH_TAFSIR.exists(), "=", PATH_TAFSIR)

df_tafsir = pd.read_csv(PATH_TAFSIR)
print("Shape:", df_tafsir.shape)
print("Kolom:", df_tafsir.columns.tolist())
df_tafsir.head()


File ada? True = /content/drive/MyDrive/fp_quran/tafsir_clean.csv
Shape: (6236, 6)
Kolom: ['surah', 'ayah', 'arabic_text', 'indonesian_translation', 'tafsir', 'tafsir_id']


,surah,ayah,arabic_text,indonesian_translation,tafsir,tafsir_id
0,Al-Fātiḥah,1,بِسْمِ اللّٰهِ الرَّحْمٰنِ الرَّحِيْمِ,Dengan nama Allah Yang Maha Pengasih lagi Maha...,Aku memulai bacaan Al-Qur'an dengan menyebut n...,Al-Fātiḥah :1
1,Al-Fātiḥah,2,اَلْحَمْدُ لِلّٰهِ رَبِّ الْعٰلَمِيْنَۙ,"Segala puji bagi Allah, Tuhan1) semesta alam",Segala puji kita persembahkan hanya untuk Alla...,Al-Fātiḥah :2
2,Al-Fātiḥah,3,الرَّحْمٰنِ الرَّحِيْمِۙ,"Yang Maha Pengasih lagi Maha Penyayang,","Dialah Yang Maha Pengasih, Pemilik dan sumber ...",Al-Fātiḥah :3
3,Al-Fātiḥah,4,مٰلِكِ يَوْمِ الدِّيْنِۗ,Pemilik hari Pembalasan.2),Dialah satu-satunya Pemilik hari Pembalasan da...,Al-Fātiḥah :4
4,Al-Fātiḥah,5,اِيَّاكَ نَعْبُدُ وَاِيَّاكَ نَسْتَعِيْنُۗ,Hanya kepada Engkaulah kami menyembah dan hany...,"Atas dasar itu semua, hanya kepada Engkaulah k...",Al-Fātiḥah :5


In [23]:
# SESUAIKAN jika nama kolom di CSV berbeda
COL_SURAH     = "surah"
COL_AYAH      = "ayah"
COL_TAFSIR_ID = "tafsir_id"
COL_TAFSIR    = "tafsir"

# Buat kolom id numerik jika belum ada
if "id" not in df_tafsir.columns:
    df_tafsir["id"] = range(1, len(df_tafsir) + 1)

COL_ID = "id"

print("Total baris:", len(df_tafsir))
df_tafsir[[COL_ID, COL_SURAH, COL_AYAH, COL_TAFSIR_ID, COL_TAFSIR]].head()


Total baris: 6236


,id,surah,ayah,tafsir_id,tafsir
0,1,Al-Fātiḥah,1,Al-Fātiḥah :1,Aku memulai bacaan Al-Qur'an dengan menyebut n...
1,2,Al-Fātiḥah,2,Al-Fātiḥah :2,Segala puji kita persembahkan hanya untuk Alla...
2,3,Al-Fātiḥah,3,Al-Fātiḥah :3,"Dialah Yang Maha Pengasih, Pemilik dan sumber ..."
3,4,Al-Fātiḥah,4,Al-Fātiḥah :4,Dialah satu-satunya Pemilik hari Pembalasan da...
4,5,Al-Fātiḥah,5,Al-Fātiḥah :5,"Atas dasar itu semua, hanya kepada Engkaulah k..."


In [24]:
# ==== GANTI SESUAI ORANG/NOTEBOOK ====
PART = 1          # orang ke-1: 1, ke-2: 2, ... sampai 5
NUM_PARTS = 5     # jumlah total pembagian (5 Colab)
# =====================================

total_rows = len(df_tafsir)

rows_per_part = total_rows // NUM_PARTS
sisa = total_rows % NUM_PARTS

# Distribusi: PART 1..sisa dapat (rows_per_part+1), sisanya dapat rows_per_part
if PART <= sisa:
    start_idx = (PART - 1) * (rows_per_part + 1)
    end_idx = start_idx + (rows_per_part + 1)
else:
    start_idx = sisa * (rows_per_part + 1) + (PART - sisa - 1) * rows_per_part
    end_idx = start_idx + rows_per_part

print(f"TOTAL baris : {total_rows}")
print(f"PART {PART}/{NUM_PARTS} → rows {start_idx} : {end_idx}")

df_part = df_tafsir.iloc[start_idx:end_idx].reset_index(drop=True)
print("Shape df_part:", df_part.shape)
df_part[[COL_ID, COL_SURAH, COL_AYAH, COL_TAFSIR_ID, COL_TAFSIR]].head()


TOTAL baris : 6236
PART 1/5 → rows 0 : 1248
Shape df_part: (1248, 7)


,id,surah,ayah,tafsir_id,tafsir
0,1,Al-Fātiḥah,1,Al-Fātiḥah :1,Aku memulai bacaan Al-Qur'an dengan menyebut n...
1,2,Al-Fātiḥah,2,Al-Fātiḥah :2,Segala puji kita persembahkan hanya untuk Alla...
2,3,Al-Fātiḥah,3,Al-Fātiḥah :3,"Dialah Yang Maha Pengasih, Pemilik dan sumber ..."
3,4,Al-Fātiḥah,4,Al-Fātiḥah :4,Dialah satu-satunya Pemilik hari Pembalasan da...
4,5,Al-Fātiḥah,5,Al-Fātiḥah :5,"Atas dasar itu semua, hanya kepada Engkaulah k..."


In [25]:
def generate_queries_from_tafsir(tafsir_text: str, max_n: int = 8):
    """
    Menghasilkan 3 sampai max_n query Bahasa Indonesia dari satu teks tafsir.
    Return: (list[str], usage_dict)
    """
    prompt = f"""
    Buat antara 3 sampai {max_n} pertanyaan (query) dalam Bahasa Indonesia
    berdasarkan teks tafsir berikut:

    <tafsir>
    {tafsir_text}
    </tafsir>

    Ketentuan penting:
    - Pertanyaan harus natural seperti pengguna Muslim awam yang ingin memahami isi tafsir.
    - Pertanyaan harus benar-benar relevan dengan makna tafsir, bukan keluar ke topik lain.
    - Jika makna tafsir sederhana, cukup 3–4 pertanyaan yang fokus dan jelas.
    - Jika makna tafsir luas (banyak nilai, hukum, adab, atau akidah), boleh sampai mendekati {max_n} pertanyaan.
    - Gunakan variasi bentuk tanya: "apa", "mengapa", "bagaimana", "siapa", "kapan", dsb.
    - Jangan menyebut kata 'tafsir ini menjelaskan' atau 'menurut tafsir' terlalu sering.
    - Jangan menyebut bahwa ini dataset, tugas akhir, penelitian, atau model AI.

    Batasan:
    - Pertanyaan jangan terlalu panjang (idealnya 1 kalimat).
    - Hindari menyebut nomor surah/ayat secara eksplisit kecuali memang penting.
    - Jangan menyalin teks tafsir secara mentah sebagai pertanyaan.

    Format jawaban:
    Kembalikan dalam JSON dengan format PERSIS seperti ini:

    {{
      "queries": [
        "pertanyaan 1...",
        "pertanyaan 2...",
        "pertanyaan 3..."
      ]
    }}

    Syarat tambahan:
    - Panjang array "queries" harus antara 3 sampai {max_n}.
    - Setiap elemen adalah string pertanyaan tanpa string kosong.
    """

    resp = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                "role": "system",
                "content": "Keluarkan HANYA JSON valid sesuai schema. Jangan beri teks lain."
            },
            {"role": "user", "content": prompt},
        ],
        response_format={"type": "json_object"},
        max_tokens=512,
    )

    data = json.loads(resp.choices[0].message.content)

    raw_queries = data.get("queries", [])
    clean_queries = []
    for q in raw_queries:
        q_clean = str(q).strip()
        if q_clean:
            clean_queries.append(q_clean)

    usage = {
        "prompt_tokens": getattr(resp.usage, "prompt_tokens", None),
        "completion_tokens": getattr(resp.usage, "completion_tokens", None),
        "total_tokens": getattr(resp.usage, "total_tokens", None),
    }

    return clean_queries, usage


In [26]:
contoh_tafsir = df_part.iloc[0][COL_TAFSIR]
print("Contoh tafsir (dipotong):\n", contoh_tafsir[:400], "...\n")

queries_test, usage_test = generate_queries_from_tafsir(contoh_tafsir, max_n=8)
print("Hasil queries:")
for i, q in enumerate(queries_test, 1):
    print(f"{i}. {q}")

print("\nUsage:", usage_test)


Contoh tafsir (dipotong):
 Aku memulai bacaan Al-Qur'an dengan menyebut nama Allah, nama teragung bagi satu-satunya Tuhan yang patut disembah, yang memiliki seluruh sifat kesempurnaan dan tersucikan dari segala bentuk kekurangan, Yang Maha Pengasih, Pemilik dan sumber sifat kasih Yang menganugerahkan segala macam karunia, baik besar maupun kecil, kepada seluruh makhluk, Maha Penyayang Yang tiada henti memberi kasih dan keba ...

Hasil queries:
1. Apa arti dari basmalah dalam bacaan Al-Qur'an?
2. Mengapa penting memulai pekerjaan dengan menyebut nama Allah?
3. Bagaimana menyebut nama Allah dapat memberikan keberkahan?
4. Siapa yang dimaksud dengan Tuhan yang Maha Pengasih dan Penyayang?
5. Apa manfaat mengingat Allah dalam setiap pekerjaan?

Usage: {'prompt_tokens': 593, 'completion_tokens': 90, 'total_tokens': 683}


In [12]:
contoh_tafsir = df_part.iloc[0][COL_TAFSIR]
print("Contoh tafsir:\n", contoh_tafsir[:300], "...\n")

contoh_queries = generate_queries_from_tafsir(contoh_tafsir, n=3)
print("Contoh queries:")
for i, q in enumerate(contoh_queries, 1):
    print(f"{i}. {q}")


Contoh tafsir:
 Aku memulai bacaan Al-Qur'an dengan menyebut nama Allah, nama teragung bagi satu-satunya Tuhan yang patut disembah, yang memiliki seluruh sifat kesempurnaan dan tersucikan dari segala bentuk kekurangan, Yang Maha Pengasih, Pemilik dan sumber sifat kasih Yang menganugerahkan segala macam karunia, bai ...

Contoh queries:
1. Apa maksud dari menyebut nama Allah sebelum memulai sesuatu?
2. Mengapa membaca basmalah bisa mendatangkan keberkahan dalam pekerjaan?
3. Bagaimana cara mengingat Allah bisa membantu seseorang menghindari keburukan?


In [27]:
OUT_DIR = Path("/content/drive/MyDrive/fp_quran/queries")
OUT_DIR.mkdir(parents=True, exist_ok=True)

out_path = OUT_DIR / f"queries_part{PART}.csv"
print("Output file:", out_path)

if out_path.exists():
    df_existing = pd.read_csv(out_path)
    print("File sudah ada, baris:", len(df_existing))

    processed_ids = set(df_existing["seg_id"].unique())
    all_rows = df_existing.to_dict("records")
    processed_ayat_count = len(processed_ids)
else:
    print("Belum ada file, mulai baru.")
    processed_ids = set()
    all_rows = []
    processed_ayat_count = 0

print("Ayat yang sudah diproses sebelumnya (PART ini):", processed_ayat_count)


Output file: /content/drive/MyDrive/fp_quran/queries/queries_part1.csv
Belum ada file, mulai baru.
Ayat yang sudah diproses sebelumnya (PART ini): 0


In [ ]:
MAX_QUERY_PER_SEGMENT = 8   # batas atas, model boleh hasilkan 3–8
SLEEP_SEC = 0.2             # jeda antar request, jangan dihapus biar aman rate limit

total_ayat_part = len(df_part)
print("Total ayat di PART ini:", total_ayat_part)

for _, row in tqdm(df_part.iterrows(), total=total_ayat_part):
    seg_id = row[COL_ID]

    # skip kalau sudah pernah diproses (resume)
    if seg_id in processed_ids:
        continue

    tafsir_text = str(row[COL_TAFSIR]).strip()
    if not tafsir_text:
        continue

    surah     = row[COL_SURAH]
    ayah      = row[COL_AYAH]
    tafsir_id = row[COL_TAFSIR_ID]

    try:
        queries, usage = generate_queries_from_tafsir(
            tafsir_text,
            max_n=MAX_QUERY_PER_SEGMENT,
        )
    except Exception as e:
        print(f"Error pada ID {seg_id} (Surah {surah}, Ayat {ayah}): {e}")
        time.sleep(5)  # istirahat sebentar (misal kena rate limit)
        continue

    # Simpan semua query untuk satu ayat
    for i, q in enumerate(queries, start=1):
        all_rows.append({
            "seg_id": seg_id,
            "surah": surah,
            "ayah": ayah,
            "tafsir_id": tafsir_id,
            "query_index": i,
            "query_text": q,
            "prompt_tokens": usage["prompt_tokens"],
            "completion_tokens": usage["completion_tokens"],
            "total_tokens": usage["total_tokens"],
        })

    processed_ids.add(seg_id)
    processed_ayat_count += 1

    # AUTO-SAVE tiap 10 ayat yang selesai di PART ini
    if processed_ayat_count % 10 == 0:
        df_tmp = pd.DataFrame(all_rows)
        df_tmp.to_csv(out_path, index=False)
        print(f"[AUTO-SAVE] Sampai {processed_ayat_count} ayat di PART {PART} → {out_path}")

    time.sleep(SLEEP_SEC)

# Save final setelah loop selesai
df_final = pd.DataFrame(all_rows)
df_final.to_csv(out_path, index=False)
print("Selesai PART", PART)
print("Total baris tersimpan:", len(df_final), "→", out_path)


Total ayat di PART ini: 1248


  0%|          | 0/1248 [00:00<?, ?it/s]

[AUTO-SAVE] Sampai 10 ayat di PART 1 → /content/drive/MyDrive/fp_quran/queries/queries_part1.csv
